# Flexure example

This notebook reproduces the [flexure example](https://fastscape-lem.github.io/fastscapelib-fortran/#_flexure_test_f90) provided in the fastscapelib-fortran library.

In [ ]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import fastscape

%matplotlib inline

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

## Import, customize and inspect the model

Let's start from the sediment model available in [fastscape](https://fastscape.readthedocs.io/en/latest/).

In [ ]:
from fastscape.models import sediment_model

Let's add, remove or update a few processes:

In [ ]:
from fastscape.processes import (Escarpment, Flexure, StratigraphicHorizons,
                                 StreamPowerChannelTD, TwoBlocksUplift,
                                 UniformRectilinearGrid2D, AdaptiveFlowRouter)


@xs.process
class EscarpmentWithPertubation(Escarpment):
    """Adds a small perturbation to drive flow."""
    
    y = xs.foreign(UniformRectilinearGrid2D, 'y')
    grid_length = xs.foreign(UniformRectilinearGrid2D, 'length')
    
    def initialize(self):
        super(EscarpmentWithPertubation, self).initialize()
        
        perturb = np.cos(self.y / self.grid_length[0] * 2. * np.pi)
        
        self.elevation += perturb[:, None]


In [ ]:
model = (
    sediment_model
    .update_processes({
        'init_topography': EscarpmentWithPertubation,
        'uplift': TwoBlocksUplift,
        'spl': StreamPowerChannelTD,
        'flexure': Flexure,
        'flow': AdaptiveFlowRouter,
        'strati': StratigraphicHorizons
    })
    .drop_processes('diffusion')
)

In [ ]:
model

In [ ]:
model.visualize(show_inputs=True)

## Model setup

In [ ]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'time': np.arange(0, 1e7 + 2e4, 2e4),
        'out': np.arange(0, 1e7 + 2e5, 2e5),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [201, 401],
        'grid__length': [2e5, 4e5],
        'boundary__status': ['fixed_value', 'core', 'looped', 'looped'],
        'init_topography': {
            'x_left': 2e5,
            'x_right': 2e5,
            'elevation_left': 0.,
            'elevation_right': 100.
        },
        'uplift': {
            'x_position': 2e5,
            'rate_left': 0.,
            'rate_right': 3e-4
        },
        'spl': {
            'k_coef': 1e-5,
            'g_coef': 1.,
        },
        'flexure': {
            'lithos_density': 2400.,
            'asthen_density': 3250.,
            'e_thickness': 1e4
        },
        'strati': {
            'freeze_time': np.linspace(0, 1e7, 5)
        }
    },
    output_vars={
        'topography__elevation': 'out',
        'active_layer__thickness': 'out',
        'strati__elevation': 'out',
    }
)

in_ds

## Run the model


In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=model)

In [ ]:
out_ds

## Plot the outputs

In [ ]:
import hvplot.xarray
import holoviews as hv
from xshade import hillshade


thickness_plot = out_ds.active_layer__thickness.hvplot.image(
    x='x', y='y', clim=(0, 600),
    cmap=plt.cm.viridis, groupby='out'
)

hillshade_plot = hillshade(out_ds, 'out').hvplot.image(
    x='x', y='y', cmap=plt.cm.gray, alpha=0.5,
    colorbar=False, hover=False, groupby='out'
)

yhorizons_plot = out_ds.strati__elevation.sel(y=1e5).hvplot.line(
    by='horizon', groupby='out', ylim=(-2000, 2000),
    legend='top_right',
)

hv.Layout((thickness_plot * hillshade_plot) + yhorizons_plot).cols(1)